In [58]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time

from assim_tools import *

import config as c
from conversion import *
from parallel import *


In [67]:
c.work_dir = '/Users/yueng/scratch/qg_testcase'
c.time = '200101050000'

In [108]:
state_info = parse_state_info(c)
obs_info = parse_obs_info(c)

number of unique field records, nrec=8


In [109]:
partitions = [tuple(lst) for lst in np.load(c.work_dir+'/cycle/'+c.time+'/analysis/partitions.npy', allow_pickle=True)]
obs_inds = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/obs_inds.npy', allow_pickle=True).item()
par_list = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/par_list.npy', allow_pickle=True).item()

In [110]:
c.comm.size = 1
rec_list_full = [i for i in state_info['fields'].keys()]
rec_size = np.array([2 if r['is_vector'] else 1 for i,r in state_info['fields'].items()])
rec_list = distribute_tasks(c.comm, rec_list_full, rec_size)

obs_rec_list_full = [i for i in obs_info['records'].keys()]
obs_rec_size = np.array([2 if r['is_vector'] else 1 for i,r in obs_info['records'].items()])
obs_rec_list = distribute_tasks(c.comm, obs_rec_list_full, obs_rec_size)

c.comm.size = 4
mem_list = distribute_tasks(c.comm, np.arange(c.nens))

In [111]:
c.pid_mem = 0
c.pid_rec = 0

state_prior = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/state_prior.{}.{}.npy'.format(c.pid_mem, c.pid_rec), allow_pickle=True).item()
z_state = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/z_state.{}.{}.npy'.format(c.pid_mem, c.pid_rec), allow_pickle=True).item()
lobs = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/lobs.{}.{}.npy'.format(c.pid_mem, c.pid_rec), allow_pickle=True).item()
lobs_prior = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/lobs_prior.{}.{}.npy'.format(c.pid_mem, c.pid_rec), allow_pickle=True).item()


In [122]:
pid_mem = 0
pid_rec = 0
obs_seq = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/obs_seq.{}.npy'.format(pid_rec), allow_pickle=True).item()
obs_prior_seq = np.load(c.work_dir+'/cycle/'+c.time+'/analysis/obs_prior_seq.{}.{}.npy'.format(pid_mem, pid_rec), allow_pickle=True).item()

In [ ]:
##which field record to show?
rec_id = [i for i,r in state_info['fields'].items() if r['name']=='velocity' and r['k']==0][0]

##which member?
mem_id = 0

binfile = '/Users/yueng/scratch/qg_testcase/cycle/200101050000/analysis/prior_mean_state.bin'
fld0 = read_field(binfile, state_info, c.mask, mem_id, rec_id)
binfile = '/Users/yueng/scratch/qg_testcase/cycle/200101050000/analysis/post_mean_state.bin'
fld1 = read_field(binfile, state_info, c.mask, mem_id, rec_id)

fig, ax = plt.subplots(1, 1, figsize=(6,5))
im = c.grid.plot_field(ax, (fld1-fld0)[0,...], vmin=-4, vmax=4, cmap='bwr')
plt.colorbar(im)
c.grid.plot_land(ax, color='gray', linecolor='k')

##obs network
#ax.scatter(obs_seq[0]['x'], obs_seq[0]['y'], 5, color='k', marker='.')